In [13]:
from tools.db_tools import fetch_jc_data

In [20]:

result = fetch_jc_data("how many job cards are there in the database issue wise and for each issue provide the count for model wise")


SELECT issue_area, vehiclemode, COUNT(DISTINCT job_card_number) AS job_card_count FROM job_card_data GROUP BY issue_area, vehiclemode ORDER BY issue_area, job_card_count DESC;


In [21]:
print(result)

[('accessory_belt_drive_system', None, 58), ('accessory_belt_drive_system', 'PRO 3019', 14), ('accessory_belt_drive_system', 'PRO 2095XP Truck', 13), ('accessory_belt_drive_system', 'PRO 8035T', 13), ('accessory_belt_drive_system', 'PRO 2110', 8), ('accessory_belt_drive_system', 'PRO 2049 Truck', 7), ('accessory_belt_drive_system', 'PRO 2059XP', 6), ('accessory_belt_drive_system', 'PRO 3019S', 6), ('accessory_belt_drive_system', 'PRO 3010 cwc', 4), ('accessory_belt_drive_system', 'PRO 6028', 3), ('accessory_belt_drive_system', 'PRO 6048', 2), ('accessory_belt_drive_system', 'PRO 3015XP', 2), ('accessory_belt_drive_system', 'PRO 2080T', 2), ('accessory_belt_drive_system', 'PRO 2114XP CNG', 2), ('accessory_belt_drive_system', 'PRO 6037', 2), ('accessory_belt_drive_system', 'PRO 3010 Bus', 2), ('accessory_belt_drive_system', 'PRO 6016 cwc', 2), ('accessory_belt_drive_system', 'PRO 1095XP Truck', 1), ('accessory_belt_drive_system', 'PRO 8055', 1), ('accessory_belt_drive_system', 'PRO 2112 

In [ ]:
[('0111222015', 'T/M ASSY...OVERHAUL', 'LABOR VALUE', 140), ('ID200359', 'ID200359, BEARING PILOT MAIN SHAFT - HCV', 'SPARES', 85), ('0104211110', 'CLUTCH DISC...REPLACEMENT', 'LABOR VALUE', 76), ('0111221026', '9 S T/M ASSYOVERHAUL VECV', 'LABOR VALUE', 48), ('0104211120', 'CLUTCH COVER ASSY...REPLACEMENT', 'LABOR VALUE', 46), ('0111221124', 'T/M Assy. Top Cover….Overhaul', 'LABOR VALUE', 46), ('ME603242', 'ME603242, RING SYNCHRONIZER', 'SPARES', 46), ('ID379580', 'ID379580, OIL SEAL, M/T CASEEXTN., RR -', 'SPARES', 45), ('ID380903', 'CLUTCH DISC DIA 310', 'SPARES', 42), ('ID360785', 'COVER ASSY. CLUTCH', 'SPARES', 39)]


## `data Prep`

In [15]:
import pandas as pd 

data = pd.read_parquet('/Users/gauthambalraj/Dev/VECV/Pixel/data/jc_data.parquet')

In [5]:
pd.set_option('display.max_columns',None)

In [9]:
excludables = pd.read_csv("/Users/gauthambalraj/Dev/VECV/Track-7/phaseQ/DATA/Excludables.csv")

In [ ]:
data.columns

In [ ]:
import warnings
import pandas as pd

def process_parts(df, excludables, debug=True):

    # preserve original pandas warning setting
    pd_warning_setting = pd.options.mode.chained_assignment
    pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning
    warnings.filterwarnings('ignore', category=FutureWarning)

        # Step 1: Merge df with excludables
    if debug:
        print("Step 1: Merging df with excludables")
    merged = df.merge(
        right=excludables,
        how="left",
        left_on="jc_item_material",
        right_on="Material Code"
    )
    merged["is_excludable_spare"].fillna("N", inplace=True)

    # Step 2: Apply prefix rules
    if debug:
        print("Step 2: Applying prefix-based excludable logic")
    merged['jc_item_material'].fillna('', inplace=True)
    merged.loc[
        merged['jc_item_material'].str.startswith(('LP', 'IZ', 'IM')),
        'is_excludable_spare'
    ] = 'Y'

    return merged


        
data = process_parts(data, excludables, debug=True)


In [19]:
jc_data = data[data['is_excludable_spare']=='N']

In [ ]:
jc_data['job_card_number'].nunique()

In [32]:
jc_data.to_parquet("/Users/gauthambalraj/Dev/VECV/Pixel/data/jc_data.parquet", index=False)

In [ ]:
# How many unique job cards do we have?
total_unique = jc_data['job_card_number'].nunique()
print(f"Total unique job cards: {total_unique}")

# Get the first 10 000 unique job_card_numbers
first_10k = jc_data['job_card_number'].drop_duplicates().iloc[:10_000]

# Subset the DataFrame to only those 10 000 cards
subset = jc_data[jc_data['job_card_number'].isin(first_10k)]

# Check
print(f"Subset shape: {subset.shape}")
print(f"Unique cards in subset: {subset['job_card_number'].nunique()}")

In [ ]:
subset.columns

In [29]:
subset.drop(columns=[ 'is_excludable_spare','Material Code', 'Material Description', 'aggregate'],inplace=True)

In [ ]:
subset.head()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, BigInteger, Text, DateTime, Float, String
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

user = "root"
password = "Gautham@20010"
host = "localhost"
port = 3306
database = "vehicle_db"
pwd_enc = quote_plus(password)

# Your DataFrame (assuming it’s already prepared)
# data = pd.read_csv("your_file.csv")  # or however you have the DataFrame

# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{pwd_enc}@{host}:{port}/{database}")

# 3) Define MySQL column types for to_sql
dtype = {
    'job_card_number':            BigInteger(),
    'vehicle_chassis_number':     Text(),
    'vehiclemode':                String(100),
    'emission':                   String(100),
    'engine':                     String(100),
    'fuel':                       String(100),
    'enginefuel':                 String(100),
    'obd':                        Float(),
    'fert_description':           Text(),
    'customer_number':            String(100),
    'dealername':                 String(100),
    'plant':                      String(100),
    'failure_date':               DateTime(),
    'jobs_description':           Text(),
    'action_taken':               Text(),
    'causal_code':                String(100),
    'casual_description':         Text(),
    'item_type':                  String(100),
    'jc_item_material':           String(100),
    'jc_item_matrial_desription': Text(),
    'region_group':               String(100),
    'culprit_code':               String(100),
    'culprit_description':        Text(),
    'order_type_description':     Text(),
    'calendar_day':               DateTime(),
    'job_id_description':         Text(),
    'customer_voice':             Text(),
    'issue_area':                 String(100),
    'observation':                Text(),
    'parts_group':                String(100),
    'parts_aggregate':            Text(),
    'year_month_mf':              String(20),
}

# 4) Write to MySQL, replacing any existing table of the same name
subset.to_sql(
    'job_card_data',         # target table name
    con=engine,
    if_exists='replace',     # or 'append'
    index=False,
    dtype=dtype
)